## This demo app shows:
* How to run Llama2 in the cloud hosted on Replicate
* How to use LangChain to ask Llama general questions and follow up questions
* How to use LangChain to load a recent PDF doc - the Llama2 paper pdf - and chat about it. This is the well known RAG (Retrieval Augmented Generation) method to let LLM such as Llama2 be able to answer questions about the data not publicly available when Llama2 was trained, or about your own data. RAG is one way to prevent LLM's hallucination
* You should also review the [HelloLlamaLocal](HelloLlamaLocal.ipynb) notebook for more information on RAG

**Note** We will be using Replicate to run the examples here. You will need to first sign in with Replicate with your github account, then create a free API token [here](https://replicate.com/account/api-tokens) that you can use for a while. 
After the free trial ends, you will need to enter billing info to continue to use Llama2 hosted on Replicate.

Let's start by installing the necessary packages:
- sentence-transformers for text embeddings
- chromadb gives us database capabilities 
- langchain provides necessary RAG tools for this demo

And setting up the Replicate token.

In [1]:
%pip install langchain replicate sentence-transformers chromadb pypdf

  Using cached langchain-0.1.12-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-2.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
  Using cached pypdf-4.1.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached PyYAML-6.0.1-cp39-cp39-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.28-cp39-cp39-macosx_11_0_arm64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.9.3-cp39-cp39-macosx_11_0_arm64.whl.metadata (7.4 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached dataclasses_json-0.6.4-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_community-0.0.28-py3-none-any.whl.metadata (8.3 kB)
  Using cached langchain_core-0.1.32-py3-none-any.whl.metadata (6.0 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl.metadata (2.0 kB)
  Using cached numpy-1.26.4-cp39-cp39-macosx_11_

Next we call the Llama 2 model from replicate. In this example we will use the llama 2 13b chat model. You can find more Llama 2 models by searching for them on the [Replicate model explore page](https://replicate.com/explore?query=llama).

You can add them here in the format: model_name/version


In [24]:
# set up the Replicate API token
from getpass import getpass
import os

print("Getting Replicate API token...")
REPLICATE_API_TOKEN = "r8_aARpJdbqdFixYHVQ9LvCvaoRA0Svg8j2Pw5W3" 
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN
print("Replicate API token set.")

# initialize the Llama2 model

from langchain.llms import Replicate

print("Initializing Llama2 model...")
llama2_13b = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"
llm = Replicate(
    model=llama2_13b,
    model_kwargs={"temperature": 0.01, "top_p": 1, "max_new_tokens":500}
)
print("Llama2 model initialized.")


Getting Replicate API token...
Replicate API token set.
Initializing Llama2 model...
Llama2 model initialized.


With the model set up, you are now ready to ask some questions. Here is an example of the simplest way to ask the model some general questions.

In [4]:
question = "who wrote the book Innovator's dilemma?"
answer = llm.invoke(question)
print(answer)

 Hello! I'd be happy to help you with your question. The book "The Innovator's Dilemma" was written by Clayton Christensen, an American author and academic who is known for his work on innovation and disruptive technologies. He is a professor at Harvard Business School and has written several other influential books on business and leadership. Is there anything else you would like to know about this topic?


We will then try to follow up the response with a question asking for more information on the book. 

Since the chat history is not passed on Llama doesn't have the context and doesn't know this is more about the book thus it treats this as new query.


In [5]:
# chat history not passed so Llama doesn't have the context and doesn't know this is more about the book
followup = "tell me more"
followup_answer = llm(followup)
print(followup_answer)

 Hello! As a helpful assistant, I'm here to assist you with any questions or tasks you may have. Whether it's providing information on a wide range of topics, helping you with a project or task, or simply being a listening ear, I'm here to help in any way I can.

I have access to a vast amount of knowledge and resources, so if there's something you're curious about or need help with, feel free to ask! Some examples of things I can help with include:

* Answering questions on a variety of topics such as history, science, technology, health, and more
* Providing guidance on how to complete a task or project
* Offering suggestions and ideas for creative projects or brainstorming sessions
* Assisting with language translation and communication
* And much more!

Is there anything specific you would like to know or discuss? Please don't hesitate to ask, and I'll do my best to assist you.


To get around this we will need to provide the model with history of the chat. 

To do this, we will use  [`ConversationBufferMemory`](https://python.langchain.com/docs/modules/memory/types/buffer) to pass the chat history to the model and give it the capability to handle follow up questions.

In [6]:
# using ConversationBufferMemory to pass memory (chat history) for follow up questions
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

Once this is set up, let us repeat the steps from before and ask the model a simple question.

Then we pass the question and answer back into the model for context along with the follow up question.

In [7]:
# restart from the original question
answer = conversation.predict(input=question)
print(answer)

 Ah, you're asking about "The Innovator's Dilemma," the iconic book written by Clayton Christensen! He is a renowned author, professor, and business consultant known for his groundbreaking work in the fields of innovation and disruptive technologies.

Published in 1997, "The Innovator's Dilemma" explores why successful companies often struggle to adapt to new technologies and business models that ultimately disrupt their industries. The book introduces the concept of "disruptive innovation," which describes how small, unassuming ideas can eventually upend entire markets and leave established players behind.

Clayton Christensen has since become one of the most influential voices in the business world, and his ideas have been applied across various sectors, from technology and healthcare to finance and education. His follow-up books, such as "The Innovator's Solution" and "Competing Against Luck," further expand on these concepts and offer practical guidance for leaders looking to stay 

In [ ]:
# pass context (previous question and answer) along with the follow up "tell me more" to Llama who now knows more of what
memory.save_context({"input": question},
                    {"output": answer})
followup_answer = conversation.predict(input=followup)
print(followup_answer)

In [7]:
%pip install --upgrade cryptography

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 3.5 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 3.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
from langchain.document_loaders import PyPDFDirectoryLoader

DATA_PATH = 'data'  # Your root data folder path

In [18]:
loader = PyPDFDirectoryLoader(DATA_PATH)
docs = loader.load()

In [19]:
import os

# Check the number of PDF files in the data directory
pdf_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.pdf')]
print(f"Number of PDF files: {len(pdf_files)}")


Number of PDF files: 2


Next, let's explore using Llama 2 to answer questions using documents for context. 
This gives us the ability to update Llama 2's knowledge thus giving it better context without needing to finetune. 
For a more in-depth study of this, see the notebook on using Llama 2 locally [here](HelloLlamaLocal.ipynb)

We will use the PyPDFLoader to load in a pdf, in this case, the Llama 2 paper.

In [ ]:

from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("https://arxiv.org/pdf/2307.09288.pdf")
docs = loader.load()

In [ ]:
# check docs length and content
print(len(docs), docs[0].page_content[0:300])

We need to store our documents. There are more than 30 vector stores (DBs) supported by LangChain. 
For this example we will use [Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma) which is light-weight and in memory so it's easy to get started with.
For other vector stores especially if you need to store a large amount of data - see https://python.langchain.com/docs/integrations/vectorstores

We will also import the HuggingFaceEmbeddings and RecursiveCharacterTextSplitter to assist in storing the documents.

In [20]:
from langchain.vectorstores import Chroma

# embeddings are numerical representations of the question and answer text
from langchain.embeddings import HuggingFaceEmbeddings

# use a common text splitter to split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

To store the documents, we will need to split them into chunks using [`RecursiveCharacterTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter) and create vector representations of these chunks using [`HuggingFaceEmbeddings`](https://www.google.com/search?q=langchain+hugging+face+embeddings&sca_esv=572890011&ei=ARUoZaH4LuumptQP48ah2Ac&oq=langchian+hugg&gs_lp=Egxnd3Mtd2l6LXNlcnAiDmxhbmdjaGlhbiBodWdnKgIIADIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCkjeHlC5Cli5D3ABeAGQAQCYAV6gAb4CqgEBNLgBAcgBAPgBAcICChAAGEcY1gQYsAPiAwQYACBBiAYBkAYI&sclient=gws-wiz-serp) on them before storing them into our vector database. 

In general, you should use larger chuck sizes for highly structured text such as code and smaller size for less structured text. You may need to experiment with different chunk sizes and overlap values to find out the best numbers.

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(docs)

# create the vector db to store all the split chunks as embeddings
embeddings = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
)

/Users/ezra/miniforge3/envs/cloud_llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We then use ` RetrievalQA` to retrieve the documents from the vector database and give the model more context on Llama 2, thereby increasing its knowledge.

For each question, LangChain performs a semantic similarity search of it in the vector db, then passes the search results as the context to Llama to answer the question.

In [30]:
# use LangChain's RetrievalQA, to associate Llama with the loaded documents stored in the vector db
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)


question = "What does the finance act contain?"
result = qa_chain({"query": question})
print(result['result'])

 Based on the pieces of context provided, the Finance Act contains provisions related to taxation and financial regulations in Kenya. Some of the specific sections mentioned include:

* Section 38 of the Retirement Benefits Act, 1997, which relates to the exemption of funds set up exclusively for sharia-compliant investments.
* Section 31B of the Employment Act, 2007, which imposes a monthly Affordable Housing Levy on employees and employers to provide funds for affordable housing and associated social and physical infrastructure.
* Section 28 of the Unclaimed Financial Assets Act, 2011, which allows claimants to designate other persons to receive unclaimed financial assets.
* Section 20 of the Statutory Instruments Act, 2013, which provides for the review of statutory instruments.
* Section 4 of the Retirement Benefits (Deputy President and Designated State Officers) Act, 2015, which repeals certain provisions related to the retirement benefits of deputy presidents and designated stat

Now, lets bring it all together by incorporating follow up questions.

First we ask a follow up questions without giving the model context of the previous conversation. 
Without this context, the answer we get does not relate to our original question.

In [26]:
# no context passed so Llama2 doesn't have enough context to answer so it lets its imagination go wild
result = qa_chain({"query": "what are its use cases?"})
print(result['result'])

 Based on the provided context, here are some potential use cases for the various categories of items listed:

1. Discs, tapes, solid-state non-volatile storage devices, "smartcards" and other media for the recording of sound or of other phenomena: These could be used for storing and playing back audio and video content, such as music, movies, and other forms of digital media.
2. Matrices and masters for the production of discs: These would be used in the manufacturing process for creating the storage media mentioned above.
3. Royalties paid to a non-resident person by a company undertaking the manufacture of human vaccines: This could be a payment made to the owner of intellectual property (such as a patent or trademark) that is being used in the manufacture of the vaccines.
4. Interest paid to a resident person or non-resident person by a company undertaking the manufacture of human vaccines: This could be interest paid on loans or investments made in the company.
5. Investment incom

As we did before, let us use the `ConversationalRetrievalChain` package to give the model context of our previous question so we can add follow up questions.

In [27]:
# use ConversationalRetrievalChain to pass chat history for follow up questions
from langchain.chains import ConversationalRetrievalChain
chat_chain = ConversationalRetrievalChain.from_llm(llm, vectordb.as_retriever(), return_source_documents=True)

In [28]:
# let's ask the original question "What is llama2?" again
result = chat_chain({"question": question, "chat_history": []})
print(result['answer'])

 The text you provided does not contain the word "ll


In [29]:
# this time we pass chat history along with the follow up so good things should happen
chat_history = [(question, result["answer"])]
followup = "what are its use cases?"
followup_answer = chat_chain({"question": followup, "chat_history": chat_history})
print(followup_answer['answer'])

 I don't know the answer to that question. Llama2 is not a term I am familiar with, and I cannot provide information on its typical use cases.


Further follow ups can be made possible by updating chat_history.

Note that results can get cut off. You may set "max_new_tokens" in the Replicate call above to a larger number (like shown below) to avoid the cut off.

```python
model_kwargs={"temperature": 0.01, "top_p": 1, "max_new_tokens": 1000}
```

In [ ]:
# further follow ups can be made possible by updating chat_history like this:
chat_history.append((followup, followup_answer["answer"]))
more_followup = "what tasks can it assist with?"
more_followup_answer = chat_chain({"question": more_followup, "chat_history": chat_history})
print(more_followup_answer['answer'])

